In [2]:
import pandas as pd
import os
import requests
from PIL import Image
import warnings
import shutil

warnings.filterwarnings("ignore")

In [11]:
#read full training set csv, assign column names and index, save for future. Do not run this cell again
df3 = pd.read_csv('data\Train_GCC-training.tsv', sep='\t', header=None, names=['caption', 'link'])
df3.to_csv(r'meta_data\Train_GCC_indices',index_label='index')

In [8]:
#read full training set csv, assign column names and index, save for future. Do not run this cell again
df3_ob = pd.read_csv(r'data\meta_data\Image_Labels_Subset_Train_GCC-Labels-training.tsv', sep='\t', header=None, names=['caption', 'link'])
df3_ob.to_csv(r'meta_data\Image_Labels_Subset_Train_GCC_indices',index_label='index')

FileNotFoundError: [Errno 2] No such file or directory: 'data\\meta_data\\Image_Labels_Subset_Train_GCC-Labels-training.tsv'

In [5]:
train_gcc_meta = pd.read_csv(r'meta_data/Train_GCC_indices')

In [9]:
#save the current working directory to use later
cwd = os.getcwd()
cwd

'C:\\Users\\apra\\Desktop\\SPRING 2021\\DS 4400\\ML1_Final_Proj'

In [52]:
os.chdir(cwd)

In [54]:
#we start a dataframe with indices of already sampled data to keep track of the data we already have. This does not include includeabs
#initial 10k samples since the indices were lost.Do not run this cell again

initial_sample = train_gcc_meta.sample(10)
os.mkdir(r'data/small_sample')
os.mkdir(r'data/small_sample/images')
os.chdir(r'data/small_sample/images')

#keep track of number of samples sucessfully taken
num_sampled = 0

successfully_sampled_indices = []
dead_link_indices = []

#faster to append to directory list than directly dataframe
cleaned_sample = []

for idx, row in initial_sample.iterrows():
    url = row['link']
    try:
        r = requests.get(url, verify=False, timeout=2, stream=True)
        if r.status_code == 200:
            #append the row to the meta_dataframe for this sample
            cleaned_sample.append(row.to_dict())
            with open(str(idx)+'.png', 'wb') as out_file:
                #download the image and name it with its index in train.gcc
                shutil.copyfileobj(r.raw, out_file)
                #store the successfully sampled image's index
                successfully_sampled_indices.append(idx)
                print(idx)
                num_sampled +=1
        else:
            print(f"failed to download image: {idx}")
            dead_link_indices.append(idx)
            
    except requests.exceptions.ReadTimeout:
        print (f"Read timed out for index: {idx}")
    except Exception as e:
        print(e)
        print (f"Other requests excpetion for index: {idx}")

#save the dataframes

#samples metadata in parent
os.chdir('..')
sample_meta_df = pd.DataFrame.from_dict(cleaned_sample)
sample_meta_df = sample_meta_df.set_index('index')
sample_meta_df.to_csv('meta_data')



#global sampled indices and dead links dataframe in project meta_data folder
os.chdir(cwd)
os.chdir(r'meta_data')
successfully_sampled_indices_df = pd.Series(successfully_sampled_indices)
successfully_sampled_indices_df.to_csv('sampled_idx_list',index = False, header = ['index'])

dead_link_indice_df = pd.Series(dead_link_indices)
dead_link_indice_df.to_csv('dead_link_idx_list', index=False, header = ['index'])


#go back to cwd
os.chdir(cwd)



        





2329432
2180312
1327415
1255841
2678725
2389256
2927249
3286362
3131468
failed to download image: 2764134


In [55]:
print(num_sampled)
os.getcwd()

9


'C:\\Users\\apra\\Desktop\\SPRING 2021\\DS 4400\\ML1_Final_Proj'

In [56]:
def sample_n_new_images(n, to_dirname, meta_df, success_file_path, dead_file_path):
    """
    Samples and downloads n previously unsampled images from links in meta_df and saves them and their meta_data to a directory 
    with name to_dirname .Uses and updates successfully sampled indices and dead link indices in success_file_path and dead_file_path.
    
    """
    cwd = os.getcwd()
    #make directories to save sampled data
    to_dir = os.path.join(r'data',to_dirname)
    os.mkdir(to_dir)
    to_dir_images = os.path.join(to_dir,'images')
    os.mkdir(to_dir_images)
    

    
    successfully_sampled_indices = pd.read_csv(success_file_path, header = 0)
    dead_link_indices = pd.read_csv(dead_file_path, header = 0)

    
    #filter full dataframe to exclude above indices
    
    new_links = meta_df.drop(labels = successfully_sampled_indices['index'], axis='index')
    new_alive_links = new_links.drop(labels =  dead_link_indices['index'], axis = 'index')
    
    
    #randomly shuffle the filtered datframe
    num_good_rows = len(new_alive_links)
    n_sample = new_alive_links.sample(n=num_good_rows)
    
    num_successfully_sampled = 0
    #to update the indice lists for successfull and failed
    success_indices = successfully_sampled_indices['index'].tolist()
    dead_indices = dead_link_indices['index'].tolist()
    
    
    #sample metadata
    cleaned_sample = []
    
    #change directory to images directory
    os.chdir(to_dir_images)
    for idx, row in n_sample.iterrows():
        #break loop if we hit the number we want
        if(num_successfully_sampled == n):
            break
        url = row['link']
        try:
            r = requests.get(url, verify=False, timeout=2, stream=True)
            if r.status_code == 200:
                #append the row to the meta_dataframe for this sample
                cleaned_sample.append(row.to_dict())
                with open(str(idx)+'.png', 'wb') as out_file:
                    #download the image and name it with its index in train.gcc
                    shutil.copyfileobj(r.raw, out_file)
                    #store the successfully sampled image's index
                    success_indices.append(idx)
                    num_successfully_sampled +=1
            else:
                print(f"failed to download image: {idx}")
                dead_indices.append(idx)

        except requests.exceptions.ReadTimeout:
            print (f"Read timed out for index: {idx}")
            continue
        except Exception as e:
            print(e)
            print (f"Other requests excpetion for index: {idx}")
    
    
    #add newly successful and failed links to respectful dataframes
    
            
    print(f"successfully sampled {num_successfully_sampled} new indices")
    
    #save the dataframes

    #samples metadata in parent
    os.chdir('..')
    sample_meta_df = pd.DataFrame.from_dict(cleaned_sample)
    sample_meta_df = sample_meta_df.set_index('index')
    sample_meta_df.to_csv('meta_data')



    #global sampled indices and dead links dataframe in project meta_data folder
    os.chdir(cwd)
    os.chdir(r'meta_data')
    
    
    
    
    
    successfully_sampled_indices_df = pd.Series(success_indices)
    successfully_sampled_indices_df.to_csv('sampled_idx_list',index = False, header = ['index'])

    dead_link_indice_df = pd.Series(dead_indices)
    dead_link_indice_df.to_csv('dead_link_idx_list', index=False, header = ['index'])


    #go back to cwd
    os.chdir(cwd)
    
        
        
    
    
    
    
    
    
    
    
    
    
    
    

In [58]:
sample_n_new_images(20000, r'20k_sample', train_gcc_meta, r'meta_data\sampled_idx_list', r'meta_data\dead_link_idx_list')

Read timed out for index: 2850332
failed to download image: 3296198
failed to download image: 964465
failed to download image: 2303140
failed to download image: 1984064
failed to download image: 373381
failed to download image: 240776
failed to download image: 431099
failed to download image: 3167328
failed to download image: 17323
failed to download image: 1667737
failed to download image: 258940
failed to download image: 2579956
failed to download image: 134322
failed to download image: 2265596
failed to download image: 410342
HTTPSConnectionPool(host='s3st0.openlistings.com', port=443): Max retries exceeded with url: /images/properties/v2/59bb01f171a6096e3c1fc3e1/x/400/59bb0b7080d4b12bee5376f8.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C6012C96A0>, 'Connection to s3st0.openlistings.com timed out. (connect timeout=2)'))
Other requests excpetion for index: 940743
failed to download image: 896276
failed to download image: 2268109
failed to 

failed to download image: 454687
Read timed out for index: 2701010
failed to download image: 129240
failed to download image: 2081145
failed to download image: 1963895
failed to download image: 3166027
failed to download image: 2523425
failed to download image: 1954382
failed to download image: 843707
failed to download image: 1323604
failed to download image: 997858
failed to download image: 2031790
failed to download image: 1640316
failed to download image: 1685039
failed to download image: 125959
failed to download image: 1978365
failed to download image: 546098
Read timed out for index: 918971
failed to download image: 1716696
HTTPSConnectionPool(host='d1alt1wkdk73qo.cloudfront.net', port=443): Max retries exceeded with url: /images/guide/46778f262df548fc805e09095a0d082b/640x960.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C6012C97F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Other requests excpetion for i

failed to download image: 2815898
failed to download image: 1134493
failed to download image: 2267324
failed to download image: 2564242
failed to download image: 1553561
failed to download image: 1600892
failed to download image: 2921426
failed to download image: 2851117
failed to download image: 2203434
Read timed out for index: 2838272
HTTPSConnectionPool(host='images.mapsofindia.com', port=443): Max retries exceeded with url: /india-tour/newvolume/mapindia/india-tour/wp-content/blogs.dir/5/files/2012/09/hidimba-devi-temple-manali-porch.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C60125BB20>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Other requests excpetion for index: 883815
Read timed out for index: 2762645
failed to download image: 2470359
failed to download image: 1090962
Read timed out for index: 2099183
HTTPConnectionPool(host='www.hvacandnaturalgaspiping.com', port=80): Max retries exceeded with url: 

Read timed out for index: 376728
failed to download image: 1483779
failed to download image: 1139865
failed to download image: 2877254
failed to download image: 919385
failed to download image: 337210
Read timed out for index: 2432696
Read timed out for index: 3086088
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Other requests excpetion for index: 2335981
failed to download image: 417803
failed to download image: 2069038
failed to download image: 336273
failed to download image: 3095799
failed to download image: 1897702
Read timed out for index: 1641973
Read timed out for index: 559733
failed to download image: 887890
failed to download image: 1649862
failed to download image: 1630027
failed to download image: 432142
failed to download image: 838270
failed to download image: 2801828
failed to download image: 2050923
failed to download image: 1705491
failed to download image: 3118641
failed to download image: 230426
failed to download imag

failed to download image: 3288195
failed to download image: 46819
failed to download image: 2619735
failed to download image: 1093462
HTTPSConnectionPool(host='media.mnn.com', port=443): Max retries exceeded with url: /assets/images/2017/06/yellowstone-grizzly-bear-family.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C601272A90>, 'Connection to media.mnn.com timed out. (connect timeout=2)'))
Other requests excpetion for index: 326599
HTTPSConnectionPool(host='d1alt1wkdk73qo.cloudfront.net', port=443): Max retries exceeded with url: /images/guide/86bddc864a9d47328a9191259599d812/640x960.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C6012C93A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Other requests excpetion for index: 407386
failed to download image: 3053106
failed to download image: 1922852
Read timed out for index: 72525
failed to download image: 2793756
failed to d

...

failed to download image: 188061
failed to download image: 1346100
failed to download image: 1586577
failed to download image: 1345641
failed to download image: 1623709
failed to download image: 2773879
failed to download image: 630747
Read timed out for index: 2882088
failed to download image: 241290
failed to download image: 2227497
HTTPSConnectionPool(host='smhttp-ssl-41526.nexcesscdn.net', port=443): Max retries exceeded with url: /media/catalog/product/cache/1/image/2000x/9df78eab33525d08d6e5fb8d27136e95/c/1/c13_median-and-frontal-section-of-the-head.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C6012AA220>, 'Connection to smhttp-ssl-41526.nexcesscdn.net timed out. (connect timeout=2)'))
Other requests excpetion for index: 3099927
failed to download image: 1909032
failed to download image: 2751331
failed to download image: 1895620
failed to download image: 3055116
failed to download image: 1467557
failed to download image: 3075051
failed 

failed to download image: 2597077
Read timed out for index: 2696072
failed to download image: 1783746
Read timed out for index: 1543240
HTTPSConnectionPool(host='media.mnn.com', port=443): Max retries exceeded with url: /assets/images/2017/05/AmericanLadyButterflyOnPinkFlower.jpg.696x0_q80_crop-smart.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C6012547F0>, 'Connection to media.mnn.com timed out. (connect timeout=2)'))
Other requests excpetion for index: 2511632
failed to download image: 2011641
failed to download image: 506648
failed to download image: 1507998
failed to download image: 2089517
failed to download image: 3003909
failed to download image: 254213
Read timed out for index: 42454
failed to download image: 2452462
failed to download image: 1750780
failed to download image: 637921
failed to download image: 52489
failed to download image: 3083397
failed to download image: 2656297
HTTPSConnectionPool(host='cdn.images.yourquote.in', po

failed to download image: 1448274
failed to download image: 1086475
failed to download image: 365065
failed to download image: 534712
failed to download image: 611257
failed to download image: 714368
failed to download image: 1631507
Read timed out for index: 746321
failed to download image: 3240170
failed to download image: 555845
failed to download image: 2745029
failed to download image: 3269282
failed to download image: 1126739
failed to download image: 2149474
failed to download image: 1529639
failed to download image: 236052
Read timed out for index: 2052110
failed to download image: 666423
failed to download image: 1078947
Read timed out for index: 2548282
failed to download image: 1260866
failed to download image: 971512
failed to download image: 471845
failed to download image: 2246624
failed to download image: 1274632
failed to download image: 147876
failed to download image: 1731523
HTTPConnectionPool(host='absolutefetishpictures.com', port=80): Max retries exceeded with url

failed to download image: 798480
failed to download image: 2041560
HTTPConnectionPool(host='www.weddingritz.com', port=80): Max retries exceeded with url: /files/attach/images/119858/019/247/002/3cd56ef32d23.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001C6012549D0>, 'Connection to www.weddingritz.com timed out. (connect timeout=2)'))
Other requests excpetion for index: 886195
failed to download image: 112292
failed to download image: 77030
Read timed out for index: 3196683
Read timed out for index: 621798
failed to download image: 1885325
failed to download image: 2563977
failed to download image: 1400281
Read timed out for index: 1055915
failed to download image: 783427
Read timed out for index: 1002738
HTTPSConnectionPool(host='am.cdnmob.org', port=443): Max retries exceeded with url: /androidgame_img/girls_of_war/thumbs/girls_of_war.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C601275820>: Failed t

failed to download image: 1109315
HTTPSConnectionPool(host='images1.laweekly.com', port=443): Max retries exceeded with url: /imager/u/745xauto/8959968/img_4835.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C601254D60>, 'Connection to images1.laweekly.com timed out. (connect timeout=2)'))
Other requests excpetion for index: 1771662
failed to download image: 1342380
HTTPConnectionPool(host='images.plasticboxshop.co.uk', port=80): Max retries exceeded with url: /images/products/zoom/1392753759-62703400.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001C601254EE0>, 'Connection to images.plasticboxshop.co.uk timed out. (connect timeout=2)'))
Other requests excpetion for index: 1135944
failed to download image: 1774444
Read timed out for index: 2628770
failed to download image: 2011563
failed to download image: 583730
failed to download image: 1681516
failed to download image: 935946
Read timed out for index: 2

In [87]:
sample_n_new_images(100, r'100_sample_4', train_gcc_meta, r'meta_data\sampled_idx_list', r'meta_data\dead_link_idx_list')

failed to download image: 3106171
failed to download image: 2232010
failed to download image: 1280349
HTTPConnectionPool(host='images.slideplayer.com', port=80): Read timed out.
Other requests excpetion for index: 2961963
failed to download image: 1248058
Read timed out for index: 3233887
failed to download image: 554427
successfully sampled 100 new indices


In [90]:
#now to verify the downloaded data and see if any were not downloaded properly

def verify_downloads(down_dirname):
    cwd = os.getcwd()
    
   #find dirmname
    down_dir = os.path.join(r'data',down_dirname)
    down_dir_images = os.path.join(down_dir,'images')
    down_dir_meta = os.path.join(down_dir, 'meta_data')
    cleaned_meta_path = os.path.join(down_dir, 'cleaned_meta_data')
    bad_files_path = os.path.join(down_dir, 'bad_image_indices')
   

    sample_df = pd.read_csv(down_dir_meta, index_col='index')

    good_count = 0
    badFiles = []
    badNumbers = []
    
    print(len(sample_df))


    for filename in os.listdir(down_dir_images):
        if filename.endswith('.png'):
            try:
                file_path = os.path.join(down_dir_images, filename)
                img = Image.open(file_path) # open the image file
                img.verify() # verify that it is, in fact an image
                good_count+=1
            except (IOError, SyntaxError) as e:
                number = os.path.splitext(filename)[0]
                badNumbers.append(int(number))
                badFiles.append(filename)
                print('Bad file:', filename) # print out the names of corrupt files
                

  
    for bad in badFiles:    
        os.remove(os.path.join(down_dir_images,  bad))

    
    cleaned_meta_df = sample_df.drop(labels = badNumbers, axis='index')
    cleaned_meta_df.to_csv(cleaned_meta_path)
    
    bad_indices_df = pd.Series(badNumbers)
    bad_indices_df.to_csv(bad_files_path,index = False, header = ['index'])
    
    print("Number of workable images: ", good_count)
    

In [93]:
verify_downloads(r'20k_sample')

20002
Bad file: 1006951.png
Bad file: 1007762.png
Bad file: 1010583.png
Bad file: 1012717.png
Bad file: 1013445.png
Bad file: 1018792.png
Bad file: 1022543.png
Bad file: 1024948.png
Bad file: 1029012.png
Bad file: 103089.png
Bad file: 1034828.png
Bad file: 1035169.png
Bad file: 1042929.png
Bad file: 1043065.png
Bad file: 1045449.png
Bad file: 1048066.png
Bad file: 1048214.png
Bad file: 105067.png
Bad file: 1053247.png
Bad file: 1055913.png
Bad file: 1058284.png
Bad file: 106226.png
Bad file: 1064937.png
Bad file: 1065785.png
Bad file: 1096882.png
Bad file: 1101031.png
Bad file: 1113691.png
Bad file: 1115702.png
Bad file: 1134195.png
Bad file: 1142352.png
Bad file: 1142386.png
Bad file: 1143249.png
Bad file: 1145298.png
Bad file: 1157305.png
Bad file: 1161021.png
Bad file: 1161151.png
Bad file: 1161292.png
Bad file: 1163998.png
Bad file: 1166245.png
Bad file: 1170375.png
Bad file: 1173401.png
Bad file: 1173627.png
Bad file: 1175244.png
Bad file: 1177796.png
Bad file: 1178884.png
Bad fil

Bad file: 245442.png
Bad file: 2460742.png
Bad file: 2467758.png
Bad file: 246982.png
Bad file: 2471230.png
Bad file: 2473687.png
Bad file: 2479699.png
Bad file: 2483970.png
Bad file: 2486319.png
Bad file: 2489413.png
Bad file: 2491707.png
Bad file: 2493379.png
Bad file: 2496963.png
Bad file: 2503933.png
Bad file: 2512047.png
Bad file: 2516421.png
Bad file: 2516571.png
Bad file: 2521978.png
Bad file: 2522389.png
Bad file: 2526619.png
Bad file: 2529175.png
Bad file: 2531553.png
Bad file: 2536168.png
Bad file: 2551833.png
Bad file: 2553655.png
Bad file: 2556630.png
Bad file: 2556770.png
Bad file: 2558982.png
Bad file: 2561534.png
Bad file: 2562550.png
Bad file: 256298.png
Bad file: 256382.png
Bad file: 2565449.png
Bad file: 256782.png
Bad file: 2572979.png
Bad file: 2581164.png
Bad file: 2583986.png
Bad file: 2584162.png
Bad file: 2585330.png
Bad file: 2586168.png
Bad file: 2586511.png
Bad file: 2591085.png
Bad file: 2592477.png
Bad file: 2604562.png
Bad file: 2604784.png
Bad file: 26059

Bad file: 956110.png
Bad file: 962298.png
Bad file: 963135.png
Bad file: 965910.png
Bad file: 970188.png
Bad file: 970482.png
Bad file: 97908.png
Bad file: 981153.png
Bad file: 981415.png
Bad file: 983635.png
Bad file: 984860.png
Bad file: 99360.png
[1006951, 1007762, 1010583, 1012717, 1013445, 1018792, 1022543, 1024948, 1029012, 103089, 1034828, 1035169, 1042929, 1043065, 1045449, 1048066, 1048214, 105067, 1053247, 1055913, 1058284, 106226, 1064937, 1065785, 1096882, 1101031, 1113691, 1115702, 1134195, 1142352, 1142386, 1143249, 1145298, 1157305, 1161021, 1161151, 1161292, 1163998, 1166245, 1170375, 1173401, 1173627, 1175244, 1177796, 1178884, 1187642, 1193658, 1196729, 1203132, 1214279, 1219056, 1224248, 1226395, 122879, 1233545, 1235377, 1236175, 1238872, 1248389, 1248995, 1249897, 1254416, 1254725, 1258460, 1263839, 1271336, 1273476, 1278753, 1283682, 1284864, 1286139, 1289842, 1292832, 1296876, 1299070, 1300790, 130614, 130971, 1313631, 1314615, 1324787, 1329291, 1333240, 134703, 

In [ ]:
#augmenting cleaned old samples with object lables etc- metadata, no need to do for newer samples

def augment 
